## Mean Reversion EURUSD Bot

### Imports and Data Download

In [5]:
access_token ="28b934679fba1dcc72c043d30738b71e-018ed37bd43974c5f0e68c3d4ea9a69b"
accountID = "101-001-28220236-001"

In [6]:
import pandas as pd
import numpy as np
import yfinance as yf
from apscheduler.schedulers.blocking import BlockingScheduler
from oandapyV20 import API
import oandapyV20.endpoints.orders as orders
from oandapyV20.contrib.requests import MarketOrderRequest
from oanda_candles import Pair, Gran, CandleCollector, CandleClient
from oandapyV20.contrib.requests import TakeProfitDetails, StopLossDetails
#from config import access_token, accountID

data = yf.download('EURUSD=X', start='2000-01-01')
data = data[['Close']]
data = data.dropna()

half_life = 263*4*24  # From mean-reversion-and-stationarity.ipynb

[*********************100%%**********************]  1 of 1 completed


### Calculate Strategy Parameters

In [7]:
data['MovingAvg'] = data['Close'].rolling(window=half_life).mean()
data['MovingStd'] = data['Close'].rolling(window=half_life).std()
data['Z'] = (data['Close'] - data['MovingAvg']) / data['MovingStd']
data['Position'] = -data['Z']
data['PnL'] = data['Position'].shift(1) * (data['Close'].diff())
data['Cumulative PnL'] = data['PnL'].cumsum()

### Trading Job

In [8]:
def get_candles(n):
    client = CandleClient(access_token, real=False)
    collector = client.get_collector(Pair.EUR_USD, Gran.M15)
    candles = collector.grab(n)
    return candles

In [9]:
def trading_job():
    # Fetching latest price data
    candles = get_candles(half_life) 
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', "Low"])

    # Parsing candle data for trading signals
    for i, candle in enumerate(candles):
        dfstream.loc[i] = [
            float(candle.bid.o.value),  
            float(candle.bid.c.value),
            float(candle.bid.h.value),
            float(candle.bid.l.value)
        ]

    # Calculate the rolling mean and standard deviation for the z-score
    dfstream['MovingAvg'] = dfstream['Close'].rolling(window=half_life).mean()  
    dfstream['MovingStd'] = dfstream['Close'].rolling(window=half_life).std()
    dfstream['Z'] = (dfstream['Close'] - dfstream['MovingAvg']) / dfstream['MovingStd']

    # Dropping the initial rows where the rolling window is not full
    dfstream.dropna(inplace=True)

    # Generate signals based on the z-score
    dfstream['Signal'] = 0  # Initialize all signals to 0
    dfstream.loc[dfstream['Z'] > 1, 'Signal'] = 1   # Sell signal
    dfstream.loc[dfstream['Z'] < -1, 'Signal'] = 2  # Buy signal

    last_signal = dfstream['Signal'].iloc[-1]

    client = API(access_token)

    SLTPRatio = 2
    previous_candleR = abs(dfstream['Open'].iloc[-2] - dfstream['Close'].iloc[-2])

    SLBuy = float(str(candle.bid.o))-previous_candleR
    SLSell = float(str(candle.bid.o))+previous_candleR

    TPBuy = float(str(candle.bid.o))+previous_candleR*SLTPRatio
    TPSell = float(str(candle.bid.o))-previous_candleR*SLTPRatio

    print(dfstream.iloc[:-1, :])
    print(TPBuy, " ", SLBuy, " ", TPSell, " ", SLSell)

    # Sell
    if last_signal == 1:
        mo = MarketOrderRequest(instrument="EUR_USD", units=-10000, takeProfitOnFill=TakeProfitDetails(price=TPSell).data, stopLossOnFill=StopLossDetails(price=SLSell).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

    # Buy
    elif last_signal == 2:
        mo = MarketOrderRequest(instrument="EUR_USD", units=10000, takeProfitOnFill=TakeProfitDetails(price=TPBuy).data, stopLossOnFill=StopLossDetails(price=SLBuy).data)
        r = orders.OrderCreate(accountID, data=mo.data)
        rv = client.request(r)
        print(rv)

### Executing Trades Manually

In [10]:
# MANUAL
trading_job()

# SCHEDULED
#scheduler = BlockingScheduler()
#scheduler.add_job(trading_job, 'cron', day_of_week='mon-sun', hour='00-23', minute='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59', second='0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59', start_date='2023-09-26 01:00:00')
#scheduler.start()

KeyboardInterrupt: 